In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
batch_size = 32
img_height = 180
img_width = 180
TRAINING_DIR = "../Dataset/train/"
VALIDATION_DIR = "../Dataset/validation/"

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  TRAINING_DIR,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  VALIDATION_DIR,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1788 files belonging to 5 classes.
Found 472 files belonging to 5 classes.


In [4]:
class_names = train_ds.class_names
print(class_names)


['acne', 'dry', 'normal', 'oily', 'redness']


In [5]:
import warnings
warnings.filterwarnings('ignore')

data_augmentation = keras.Sequential(
  
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [6]:
import warnings
warnings.filterwarnings('ignore')

num_classes = 5

model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
epochs=100
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/100
56/56 [==============================] - 79s 1s/step - loss: 1.0399 - accuracy: 0.6376 - val_loss: 1.5546 - val_accuracy: 0.5445
Epoch 2/100
56/56 [==============================] - 65s 1s/step - loss: 0.6988 - accuracy: 0.7528 - val_loss: 2.8273 - val_accuracy: 0.3602
Epoch 3/100
56/56 [==============================] - 63s 1s/step - loss: 0.6299 - accuracy: 0.7679 - val_loss: 3.1728 - val_accuracy: 0.2818
Epoch 4/100
56/56 [==============================] - 63s 1s/step - loss: 0.5670 - accuracy: 0.7869 - val_loss: 2.0024 - val_accuracy: 0.5381
Epoch 5/100
56/56 [==============================] - 65s 1s/step - loss: 0.5747 - accuracy: 0.7830 - val_loss: 3.1660 - val_accuracy: 0.3835
Epoch 6/100
56/56 [==============================] - 64s 1s/step - loss: 0.5656 - accuracy: 0.7864 - val_loss: 3.4135 - val_accuracy: 0.4153
Epoch 7/100
56/56 [==============================] - 64s 1s/step - loss: 0.5275 - accuracy: 0.8110 - val_loss: 2.9552 - val_accuracy: 0.4936
Epoch 8/100
5

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.

model.save('acnescan_v2.h5')

In [ ]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
new_model= tf.keras.models.load_model(filepath="acnescan_v2.h5")


In [ ]:
import numpy as np
import tkinter as tk
from tkinter import filedialog
from keras.preprocessing import image

root = tk.Tk()
root.withdraw()
uploaded = filedialog.askopenfilename()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = keras.preprocessing.image.load_img(
      path, target_size=(img_height, img_width)
  )
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])

  print(
      "This image most likely belongs to {} with a {:.2f} percent confidence."
      .format(class_names[np.argmax(score)], 100 * np.max(score))
  )
 

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('acnescan_V2.tflite', 'wb') as f:
  f.write(tflite_model)